In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time 
import timeit
from IPython.display import clear_output

In [ ]:
# Definición de la ruta donde se encuentran los archivos y lectura del archivo CSV
path_a = 'C:/Users/DELL/Documents/chrome_driver/Archivos/'
entidades = pd.read_csv(path_a+'quindio_depto.csv')
entidades

In [ ]:
# Configuración del servicio y opciones para el navegador Chrome
path = "chromedriver.exe"
service = Service(path)
options = webdriver.ChromeOptions()

In [ ]:
columnas_de_interes = ['CODIGO', 'NOMBRE', 'ENTIDAD', 'CIUDAD','RECAUDO VIGEN ACTUAL SIN FONDOS(Pesos)', 'RECAUDO VIGEN ACTUAL CON FONDOS(Pesos)', 'RECAUDO VIGEN ANTERIOR SIN FONDO(Pesos)', 'RECAUDO VIGEN ANTERIOR CON FONDO(Pesos)', 'TOTAL RECAUDO(Pesos)']
columnas_final = ['CODIGO', 'NOMBRE', 'ENTIDAD', 'CIUDAD','RECAUDO VIGEN ACTUAL SIN FONDOS(Pesos)', 'RECAUDO VIGEN ACTUAL CON FONDOS(Pesos)', 'RECAUDO VIGEN ANTERIOR SIN FONDO(Pesos)', 'RECAUDO VIGEN ANTERIOR CON FONDO(Pesos)', 'TOTAL RECAUDO(Pesos)']
periodos_interes = ["DIC A DIC - 2023", "SEP A SEP - 2023", "JUN A JUN - 2023", "MAR A MAR - 2023"]

In [ ]:
# Donde se almacenara todas las filas de las entidades por ciclo
lista_filas = []

In [ ]:
# Inicialización del navegador Chrome y navegación a la página web
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.chip.gov.co/schip_rt/index.jsf")

# Esperar hasta que se cargue el elemento y damo click para ir a la pág de filtros
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#j_idt105\:InformacionEnviada')))
pag_informacion_ciudadano = driver.find_element(By.CSS_SELECTOR, '#j_idt105\:InformacionEnviada')
pag_informacion_ciudadano.click()

# Inicio del contador de ciclos
contar_ciclos = 0

# Inicia el temporizador
inicio = timeit.default_timer()

# Iteración sobre las entidades y períodos de interés
for codigo in entidades['codigo_chip']:
    for periodo in periodos_interes: 
        contar_intentos = 0
        
        # Filtro entidad, esperar a que aparezca y llenarlo 
        while True:
            try:
                filtro_entidad = driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxEntidadCiudadano_input')
                filtro_entidad.clear()
                filtro_entidad.send_keys(codigo)
                filtro_entidad.send_keys(Keys.ENTER)
                break
            except Exception as e:
                print(f"Elemento no encontrado: {e}")
                time.sleep(4)

        # Filtro de categorias que es de elección, esperamos y seleccionamos
        while True:
            try:
                filtro_categorias = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxCategoria'))
                filtro_categorias.select_by_visible_text(". : : Seleccione : : .")
                filtro_categorias.select_by_visible_text("CUIPO - CATEGORIA UNICA DE INFORMACION DEL PRESUPUESTO ORDINARIO")
                break
            except Exception as e:
                print(f"Elemento no encontrado: {e}")
                time.sleep(4)
        
        # Filtro de periodo que es de elección, esperamos y seleccionamos    
        while True:
            try:
                trimestre = False
                contar_intentos+=1
                print(contar_intentos)
                filtro_periodo = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxPeriodo'))
                filtro_periodo.select_by_visible_text(periodo)
                break
            except Exception as e:
                print(f"Elemento no encontrado: {e}")
                time.sleep(4)
                trimestre = True
            if contar_intentos >= 5:
                contar_intentos = 0
                break
                   
        # Filtro de formulario que es de elección, esperamos y seleccionamos
        if not trimestre:
            while True:
                try:
                    print(filtro_periodo)
                    filtro_formulario = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxForma'))
                    filtro_formulario.select_by_index(2)
                    break
                except Exception as e:
                    print(f"Elemento no encontrado: {e}")
                    time.sleep(4)

            # Boton generar, para obtener la tabla
            while True:
                try:
                    boton_consultar = driver.find_element(By.CSS_SELECTOR, '#frm1\:BtnConsular')
                    boton_consultar.click()
                    break
                except Exception as e:
                    print(f"Error: {e}")
                    time.sleep(4)

            # Elegir el nivel para que me muestre toda la tabla en esa página
            while True:
                try:
                    nivel = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxNivel'))
                    nivel.select_by_index(8)
                    break
                except Exception as e:
                    print(f"Elemento no encontrado: {e}")
                    time.sleep(4)

            # Extraemos html de la tabla y convertimos a un objeto bs
            while True:
                tabla_html = driver.find_element(By.CSS_SELECTOR, '#frm1\:j_idt225').get_attribute("outerHTML")
                soup = BeautifulSoup(tabla_html, "html.parser")
                filas = soup.find_all('tr')
                if len(filas) > 3:
                    break

            # Extraemos columnas y filas
            headers = [th.text.strip() for th in soup.find_all('th')]
            rows = []
            for tr in soup.find_all('tr'):
                cells = [td.text.strip() for td in tr.find_all('td')]
                rows.append(cells)

            # Creamos dataframe para ver como salieron los datos con las columnas y filas
            df_temp = pd.DataFrame(rows, columns=headers)
            df_temp = df_temp.drop(0)
            df_temp['ENTIDAD'] = codigo
            df_temp["CIUDAD"] = entidades.loc[entidades["codigo_chip"] == codigo, "Ciudad"].values[0]

            print(df_temp.shape)
            
            # Revisamos si el el daframe cuenta con las columnas que queremos
            columnas_dataframe = set(df_temp.columns)
            columnas_de_interes = set(columnas_de_interes)
            columnas_faltantes = columnas_de_interes - columnas_dataframe
            print(columnas_faltantes)

            if not columnas_faltantes:
                df_temp = df_temp[columnas_final]
            else:
                for columna in columnas_faltantes:
                    df_temp[columna] = np.nan
                df_temp = df_temp[columnas_final]

            lista_filas.extend(df_temp[columnas_final].to_numpy().tolist())

            # Volvemos a la página para generar otra tabla
            boton_volver = driver.find_element(By.CSS_SELECTOR, '#frm1\:j_idt233')
            boton_volver.click()
            
            contar_ciclos+=1
            clear_output() 
    
driver.quit()
clear_output() 

# Fin del temporizador
fin = timeit.default_timer()
tiempo_ejecucion = fin - inicio
print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")
print(f'Hizo {contar_ciclos} ciclos')


In [ ]:
# Con todas las filas extraidas, genereamos el dataframe final donde almacenara todo
df_final = pd.DataFrame(lista_filas, columns=columnas_final)
df_final

In [ ]:
df_final.to_csv("dos_entidades_v2.csv")